In [1]:
import sys
sys.path.insert(0, "../torchdriveenv")

In [2]:
import time
import torch
import torchdriveenv
from torch.utils.data import DataLoader

from datasets import TorchDriveEnvEpisodeDataset
from diffusion import DiffusionModel
from model import Resnet, EBMDiffusionModel

/home/kezhang/work/stochastic-critic/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Commercial access denied and fallback to check for academic access.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data_dir = "../torchdriveenv/examples/offline_datasets/episode_data_20241025-1300"

In [5]:
dataset = TorchDriveEnvEpisodeDataset(data_dir=data_dir)

/home/kezhang/work/stochastic-critic/.venv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [6]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [7]:
# len(dataset)

In [8]:
batch_size = 64
data_dim = dataset.state_dim

EMA = .999

n_steps = 100
net_params = {"n_layers": 4,
              "h_dim": 128,
              "emb_dim": 32}

In [9]:
net = Resnet(n_steps=n_steps, n_layers=4, x_dim=data_dim, h_dim=128, emb_dim=32)

ebm_net = EBMDiffusionModel(net)

ddpm = DiffusionModel(data_dim, n_steps, ebm_net, device=device, var_type="beta_forward").to(device)

In [10]:
def logp_unnorm(net, ddpm, x, t):
    scale_e = ddpm.energy_scale(-2 - t)
    t = torch.ones((x.shape[0],)).int() * t
    return -net.neg_logp_unnorm(x, t) * scale_e

In [11]:
losses = []
test_logpx = []
itr = 0

ddpm.train()
optimizer = torch.optim.Adam(ddpm.parameters(), lr=1e-3)


num_steps = 200000
for itr in range(num_steps):
    ddpm.train()
    
    x = next(iter(dataloader))

    start_time = time.time()

    loss = ddpm.loss(x)
    loss = loss.mean()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    duration_update = time.time() - start_time

    if itr % 100 == 0:
        print(itr, loss, "time:", duration_update)
        losses.append(loss)
        
    if itr % 1000 == 0:
        ddpm.eval()
        x_samp = ddpm.sample(n=1)
        print(x_samp)

0 tensor(83.2803, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.517838716506958
tensor([[ 0.1241,  0.1973, -0.1353, -0.0479, -0.0499,  0.1155,  0.2146, -0.0095,
          0.0555,  0.0545, -0.1298,  0.1041, -0.3090, -0.2400, -0.0218,  0.2947,
          0.2555,  0.1012, -0.1128, -0.0114, -0.0015,  0.3408,  0.1530, -0.1586,
         -0.3629,  0.1426,  0.4925, -0.1364,  0.4146,  0.1697,  0.3051,  0.1917,
          0.1234, -0.0082, -0.0721, -0.0318,  0.0592,  0.0818,  0.0689, -0.0468,
         -0.0413, -0.0547, -0.1607,  0.0319,  0.3453, -0.4914, -0.1680, -0.1860,
         -0.2659,  0.0931, -0.0768,  0.1072, -0.0902,  0.2697, -0.1559, -0.1546,
         -0.5598, -0.3917, -0.2506, -0.1345,  0.0370, -0.2540, -0.2423,  0.3161,
         -0.2175, -0.1017, -0.1508, -0.3399,  0.4850,  0.1294, -0.1180,  0.1722,
         -0.1080, -0.1212, -0.1458, -0.2237,  0.0720,  0.0481,  0.1952, -0.2583,
         -0.2846, -0.5164,  0.2430, -0.0107, -0.3245, -0.0199,  0.0140, -0.0563,
    

2100 tensor(0.5333, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0058133602142333984
2200 tensor(0.4976, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0058062076568603516
2300 tensor(1.0439, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0058231353759765625
2400 tensor(0.7323, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005800724029541016
2500 tensor(1.1262, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005939960479736328
2600 tensor(0.8569, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0058002471923828125
2700 tensor(1.1127, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005827903747558594
2800 tensor(0.6264, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0059163570404052734
2900 tensor(0.8778, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005837202072143555
3000 

5100 tensor(0.4026, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005826234817504883
5200 tensor(0.8402, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005961894989013672
5300 tensor(0.5740, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005917072296142578
5400 tensor(0.8863, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005839824676513672
5500 tensor(0.7309, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005819082260131836
5600 tensor(0.7589, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005919456481933594
5700 tensor(0.6805, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0058138370513916016
5800 tensor(0.6402, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0057451725006103516
5900 tensor(1.0625, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005840301513671875
6000 ten

8100 tensor(0.2394, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005791425704956055
8200 tensor(0.5150, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005909919738769531
8300 tensor(0.3034, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005868196487426758
8400 tensor(0.2881, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005819559097290039
8500 tensor(0.2687, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005808115005493164
8600 tensor(0.5334, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006160259246826172
8700 tensor(0.2852, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0058193206787109375
8800 tensor(0.2738, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005794525146484375
8900 tensor(0.5649, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005792140960693359
9000 tens

11100 tensor(31.9461, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005812883377075195
11200 tensor(25.2510, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005838871002197266
11300 tensor(7.7374, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005967140197753906
11400 tensor(4.4291, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005849599838256836
11500 tensor(5.1437, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005940437316894531
11600 tensor(4.6199, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011222124099731445
11700 tensor(3.8091, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011288881301879883
11800 tensor(3.5829, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005875349044799805
11900 tensor(2.8051, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005989789962768555

...

185100 tensor(0.1778, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010991573333740234
185200 tensor(0.4443, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010986566543579102
185300 tensor(0.1868, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010982275009155273
185400 tensor(0.2276, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011213064193725586
185500 tensor(0.0870, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010980606079101562
185600 tensor(0.1865, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011192083358764648
185700 tensor(0.2571, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011008739471435547
185800 tensor(0.3317, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01098322868347168
185900 tensor(0.1131, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011235952377

188100 tensor(0.0673, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011252164840698242
188200 tensor(0.1092, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011069297790527344
188300 tensor(0.0399, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010975122451782227
188400 tensor(0.6471, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010996580123901367
188500 tensor(0.2956, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011147022247314453
188600 tensor(0.1643, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011042118072509766
188700 tensor(0.2721, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010972976684570312
188800 tensor(0.2052, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010967254638671875
188900 tensor(0.1516, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01098942756

191100 tensor(0.1163, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011002302169799805
191200 tensor(0.1180, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011018514633178711
191300 tensor(0.1818, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011059045791625977
191400 tensor(0.0721, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011340856552124023
191500 tensor(0.1563, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01106405258178711
191600 tensor(0.0891, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011003494262695312
191700 tensor(0.2352, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010998249053955078
191800 tensor(0.6099, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01102137565612793
191900 tensor(0.4501, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0118529796600

194100 tensor(0.0881, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011757850646972656
194200 tensor(0.0619, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011026144027709961
194300 tensor(0.0905, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011318445205688477
194400 tensor(0.0966, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01096343994140625
194500 tensor(0.6243, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010981321334838867
194600 tensor(0.1385, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01102304458618164
194700 tensor(0.4795, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011251688003540039
194800 tensor(0.3616, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010956287384033203
194900 tensor(0.1603, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0109517574310

197100 tensor(0.1084, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011023998260498047
197200 tensor(0.0495, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012033462524414062
197300 tensor(0.1465, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011385202407836914
197400 tensor(0.1696, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01106882095336914
197500 tensor(0.1037, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011005640029907227
197600 tensor(0.1948, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010987997055053711
197700 tensor(0.2615, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01140737533569336
197800 tensor(0.1991, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011019229888916016
197900 tensor(0.1171, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0110006332397